In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = 9, 6
%load_ext autoreload
%autoreload 2
import sample_data

np.random.seed(0)
N_train = 50000; N_test = 10000
N = N_train + N_test
train = np.arange(N_train); test = np.arange(N_test) + N_train
n_min = 250; n_max = 250

X, y = sample_data.periodic(N, n_min, n_max, t_max=2*np.pi, even=False,
                            A=1., sigma=2., w_min=0.1, w_max=1.)

In [ ]:
fig, ax = plt.subplots(2, 3, sharey=True)
for j in range(6):
#    i = np.where(y == j)[0][0]
    i = j
#    ax.ravel()[j].plot(np.linspace(0, n_max, n_max), X[i], 'o')
    ax.ravel()[j].plot(X[i][:, 0], X[i][:, 1], 'o')
    ax.ravel()[j].set(xlabel="t", ylabel="y")

In [ ]:
import os
import tensorflow as tf
from keras import backend as K
from keras.layers import (Input, Dense, TimeDistributed, Activation, LSTM,
                          Dropout, merge, Reshape, Flatten, RepeatVector)
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import ProgbarLogger, TensorBoard
from keras.utils.visualize_util import model_to_dot
from IPython.display import clear_output, SVG, display


lr = 2e-3
for lstm_size in [128, 256]:
    for num_layers in [3, 4]:
        for drop_frac in [0.5]:
            sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.7)))
            K.set_session(sess)

            run = "lstm_{:03d}_x{}_{:1.0e}_drop{}".format(lstm_size, num_layers, lr, drop_frac).replace('e-', 'm').replace('.', '')
            print(run)
            model = Sequential()
            model.add(LSTM(lstm_size, input_shape=(n_max, 2), return_sequences=(num_layers > 1)))
            for i in range(1, num_layers):
                model.add(Dropout(drop_frac))
                model.add(LSTM(lstm_size, return_sequences=(i != num_layers - 1)))
            model.add(Dense(1, activation='linear'))
            adam = Adam(lr=lr)
            model.compile(optimizer=adam, loss='mse', metrics=[])

            model_dot = model_to_dot(model).create(prog='dot', format='svg')
            print('{} MB'.format(model.count_params() * np.dtype('float32').itemsize / 1e6))
            print(model.summary())
#            display(SVG(model_dot))
            
            log_dir = os.path.expanduser('~/Dropbox/Documents/timeflow/keras_logs/period/{}'.format(run))
            !rm -rf $log_dir
            #if os.path.exists(log_dir):
            #    raise Exception("Log directory already exists, not overwriting")
            history = model.fit(X[train], y[train], 
                                 nb_epoch=25, batch_size=1000, validation_split=0.2,
                                 callbacks=[ProgbarLogger(), TensorBoard(log_dir=log_dir, write_graph=False)])
            model.save_weights(os.path.join(log_dir, 'weights.h5'), overwrite=True)
            tf.reset_default_graph()
            clear_output()

In [ ]:
%%time
pred_keras = model.predict(X[test]).ravel()

In [ ]:
%%time
from gatspy.periodic import LombScargleFast

pred_gatspy = np.zeros(N_test)
for i in range(N_test):
    opt_args = {'period_range': (0.1, 0.9 * (X[test[i]][-1, 0] - X[test[i]][0, 0])), 'quiet': True}
    model_gatspy = LombScargleFast(fit_period=True, optimizer_kwds=opt_args, silence_warnings=True)
    pred_gatspy[i] = 1. / model_gatspy.fit(X[test[i]][:, 0], X[test[i]][:, 1]).best_period

In [ ]:
sns.jointplot(pred_keras - testY, pred_gatspy - testY)

In [ ]:
print(np.median((pred_keras - testY) ** 2))
print(np.median((pred_gatspy - testY) ** 2))